# 🔄 Task 2: Data Preprocessing and Augmentation

## 🎯 Objective
Learn and implement image preprocessing and augmentation techniques using NumPy from scratch.

---

## 📚 Theory: Image Preprocessing

### Why Preprocessing?
1. **Standardize dimensions** - Neural networks need fixed input sizes
2. **Normalize values** - Faster convergence, stable gradients
3. **Augmentation** - Increase dataset diversity, reduce overfitting

### ML Rules Applied:
- **Rule #4**: Keep the first model simple - use standard preprocessing
- **Rule #6**: Be careful about dropped data when copying pipelines
- **Rule #17**: Start with directly observed features

---

## 📐 Mathematical Foundations

### 1. Image Resizing (Bilinear Interpolation)

To resize image from (H₁, W₁) to (H₂, W₂):

```
For output pixel (i, j):
  x = i × (H₁-1)/(H₂-1)    # Map to source coordinates
  y = j × (W₁-1)/(W₂-1)
  
  x₀, x₁ = floor(x), ceil(x)   # Neighboring pixels
  y₀, y₁ = floor(y), ceil(y)
  
  # Bilinear interpolation
  f(x,y) = (1-α)(1-β)f(x₀,y₀) + α(1-β)f(x₁,y₀) + (1-α)βf(x₀,y₁) + αβf(x₁,y₁)
  
  where α = x - x₀, β = y - y₀
```

### 2. Normalization

**Min-Max Normalization (0-1 scaling):**
```
x_norm = (x - min) / (max - min)
For images: x_norm = x / 255.0
```

**Z-Score Standardization:**
```
x_std = (x - μ) / σ
where μ = mean, σ = std deviation
```

### 3. Data Augmentation Transforms

**Horizontal Flip:**
```
I_flip[i, j] = I[i, W-1-j]
```

**Rotation (θ degrees):**
```
x' = x×cos(θ) - y×sin(θ)
y' = x×sin(θ) + y×cos(θ)
```

**Brightness Adjustment:**
```
I_bright = I + β    (additive)
I_bright = I × α    (multiplicative)
```

In [ ]:
# Import Libraries
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from PIL import Image
import cv2
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-whitegrid')
print("✅ Libraries imported!")

In [ ]:
# Project paths
PROJECT_ROOT = Path(r"D:\het\SELF\RP\YOLO-V11-PRO")
RAW_DATA_DIR = PROJECT_ROOT / "data" / "raw"
PROCESSED_DIR = PROJECT_ROOT / "data" / "processed"

# Find dataset
DATASET_DIR = None
for p in [RAW_DATA_DIR / "DATASET", RAW_DATA_DIR]:
    if (p / "TRAIN").exists():
        DATASET_DIR = p
        break

if DATASET_DIR:
    TRAIN_DIR = DATASET_DIR / "TRAIN"
    print(f"✅ Dataset found: {DATASET_DIR}")
else:
    print("❌ Please download dataset first (see Task 1)")

## 🧮 NumPy Implementation: Preprocessing Functions (From Scratch!)

In [ ]:
# ============================================================
# IMAGE RESIZING - Bilinear Interpolation (NumPy from scratch)
# ============================================================

def bilinear_resize_numpy(image, new_height, new_width):
    """
    Resize image using bilinear interpolation - NUMPY ONLY.
    
    Mathematical basis:
    f(x,y) = (1-α)(1-β)f(x₀,y₀) + α(1-β)f(x₁,y₀) 
           + (1-α)βf(x₀,y₁) + αβf(x₁,y₁)
    
    Args:
        image: Input image array (H, W, C) or (H, W)
        new_height: Target height
        new_width: Target width
    
    Returns:
        Resized image array
    """
    # Get original dimensions
    if len(image.shape) == 3:
        old_h, old_w, channels = image.shape
    else:
        old_h, old_w = image.shape
        channels = 1
        image = image[:, :, np.newaxis]
    
    # Create output array
    resized = np.zeros((new_height, new_width, channels), dtype=np.float32)
    
    # Calculate scaling factors
    row_scale = (old_h - 1) / (new_height - 1) if new_height > 1 else 0
    col_scale = (old_w - 1) / (new_width - 1) if new_width > 1 else 0
    
    # Vectorized implementation for speed
    row_indices = np.arange(new_height) * row_scale
    col_indices = np.arange(new_width) * col_scale
    
    # Get floor and ceil indices
    row_floor = np.floor(row_indices).astype(int)
    row_ceil = np.minimum(np.ceil(row_indices).astype(int), old_h - 1)
    col_floor = np.floor(col_indices).astype(int)
    col_ceil = np.minimum(np.ceil(col_indices).astype(int), old_w - 1)
    
    # Calculate interpolation weights (α and β)
    row_alpha = row_indices - row_floor  # α = x - x₀
    col_beta = col_indices - col_floor   # β = y - y₀
    
    # Loop through each pixel (for clarity)
    for i in range(new_height):
        for j in range(new_width):
            r0, r1 = row_floor[i], row_ceil[i]
            c0, c1 = col_floor[j], col_ceil[j]
            alpha = row_alpha[i]
            beta = col_beta[j]
            
            # Bilinear interpolation formula
            for c in range(channels):
                resized[i, j, c] = (
                    (1 - alpha) * (1 - beta) * image[r0, c0, c] +
                    alpha * (1 - beta) * image[r1, c0, c] +
                    (1 - alpha) * beta * image[r0, c1, c] +
                    alpha * beta * image[r1, c1, c]
                )
    
    if channels == 1:
        resized = resized[:, :, 0]
    
    return resized.astype(np.uint8)

print("✅ Bilinear resize function defined (NumPy only!)")

In [ ]:
# ============================================================
# NORMALIZATION FUNCTIONS (NumPy from scratch)
# ============================================================

def normalize_minmax(image, new_min=0.0, new_max=1.0):
    """
    Min-Max normalization: scales values to [new_min, new_max].
    
    Formula: x_norm = (x - min) / (max - min) × (new_max - new_min) + new_min
    """
    img_float = image.astype(np.float32)
    img_min = np.min(img_float)
    img_max = np.max(img_float)
    
    # Avoid division by zero
    if img_max - img_min == 0:
        return np.full_like(img_float, new_min)
    
    normalized = (img_float - img_min) / (img_max - img_min)
    normalized = normalized * (new_max - new_min) + new_min
    
    return normalized

def normalize_zscore(image, mean=None, std=None):
    """
    Z-score standardization: x_std = (x - μ) / σ
    
    If mean/std not provided, calculates from image.
    """
    img_float = image.astype(np.float32)
    
    if mean is None:
        mean = np.mean(img_float)
    if std is None:
        std = np.std(img_float)
    
    # Avoid division by zero
    if std == 0:
        return np.zeros_like(img_float)
    
    standardized = (img_float - mean) / std
    return standardized

def normalize_imagenet(image):
    """
    ImageNet normalization (common for pretrained models).
    
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]
    """
    img_float = image.astype(np.float32) / 255.0
    
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    
    normalized = (img_float - mean) / std
    return normalized

print("✅ Normalization functions defined")

In [ ]:
# ============================================================
# DATA AUGMENTATION FUNCTIONS (NumPy from scratch)
# ============================================================

def horizontal_flip(image):
    """Flip image horizontally: I_flip[i,j] = I[i, W-1-j]"""
    return image[:, ::-1].copy()

def vertical_flip(image):
    """Flip image vertically: I_flip[i,j] = I[H-1-i, j]"""
    return image[::-1, :].copy()

def adjust_brightness(image, factor):
    """
    Adjust brightness by multiplicative factor.
    Formula: I_bright = I × factor
    factor > 1: brighter, factor < 1: darker
    """
    adjusted = image.astype(np.float32) * factor
    return np.clip(adjusted, 0, 255).astype(np.uint8)

def adjust_contrast(image, factor):
    """
    Adjust contrast around mean.
    Formula: I_contrast = (I - mean) × factor + mean
    """
    img_float = image.astype(np.float32)
    mean = np.mean(img_float)
    adjusted = (img_float - mean) * factor + mean
    return np.clip(adjusted, 0, 255).astype(np.uint8)

def add_gaussian_noise(image, mean=0, std=25):
    """
    Add Gaussian noise: I_noisy = I + N(μ, σ²)
    """
    noise = np.random.normal(mean, std, image.shape)
    noisy = image.astype(np.float32) + noise
    return np.clip(noisy, 0, 255).astype(np.uint8)

def rotate_90(image, k=1):
    """Rotate image by 90×k degrees counterclockwise."""
    return np.rot90(image, k)

def random_crop(image, crop_size):
    """
    Random crop of specified size.
    """
    h, w = image.shape[:2]
    crop_h, crop_w = crop_size
    
    if h < crop_h or w < crop_w:
        return image
    
    top = np.random.randint(0, h - crop_h + 1)
    left = np.random.randint(0, w - crop_w + 1)
    
    return image[top:top+crop_h, left:left+crop_w].copy()

print("✅ Augmentation functions defined (NumPy only!)")

In [ ]:
# ============================================================
# VISUALIZE AUGMENTATIONS
# ============================================================

def visualize_augmentations(image_path):
    """Show all augmentation effects on a single image."""
    
    img = np.array(Image.open(image_path))
    
    augmentations = [
        ('Original', img),
        ('Horizontal Flip', horizontal_flip(img)),
        ('Vertical Flip', vertical_flip(img)),
        ('Brightness +50%', adjust_brightness(img, 1.5)),
        ('Brightness -30%', adjust_brightness(img, 0.7)),
        ('Contrast +50%', adjust_contrast(img, 1.5)),
        ('Gaussian Noise', add_gaussian_noise(img, std=30)),
        ('Rotate 90°', rotate_90(img, 1)),
    ]
    
    fig, axes = plt.subplots(2, 4, figsize=(16, 8))
    fig.suptitle('🔄 Data Augmentation Techniques (NumPy Implementation)', fontsize=14, fontweight='bold')
    
    for ax, (title, aug_img) in zip(axes.flat, augmentations):
        ax.imshow(aug_img)
        ax.set_title(title, fontsize=10)
        ax.axis('off')
    
    plt.tight_layout()
    plt.savefig(PROJECT_ROOT / 'docs' / 'assets' / 'augmentations_demo.png', dpi=150)
    plt.show()
    print("✅ Saved to: docs/assets/augmentations_demo.png")

# Test with a sample image
if DATASET_DIR:
    sample_images = list((TRAIN_DIR / "O").glob("*.jpg"))[:1]
    if sample_images:
        visualize_augmentations(sample_images[0])

In [ ]:
# ============================================================
# VISUALIZE NORMALIZATION EFFECTS
# ============================================================

def visualize_normalization(image_path):
    """Show normalization effects with histograms."""
    
    img = np.array(Image.open(image_path))
    
    # Apply normalizations
    minmax = normalize_minmax(img)
    zscore = normalize_zscore(img)
    
    fig, axes = plt.subplots(2, 3, figsize=(15, 8))
    fig.suptitle('📊 Normalization Techniques & Distributions', fontsize=14, fontweight='bold')
    
    # Original
    axes[0, 0].imshow(img)
    axes[0, 0].set_title('Original [0-255]')
    axes[0, 0].axis('off')
    axes[1, 0].hist(img.flatten(), bins=50, color='blue', alpha=0.7)
    axes[1, 0].set_xlabel('Pixel Value')
    axes[1, 0].set_title(f'μ={img.mean():.1f}, σ={img.std():.1f}')
    
    # Min-Max
    axes[0, 1].imshow(minmax)
    axes[0, 1].set_title('Min-Max [0-1]')
    axes[0, 1].axis('off')
    axes[1, 1].hist(minmax.flatten(), bins=50, color='green', alpha=0.7)
    axes[1, 1].set_xlabel('Normalized Value')
    axes[1, 1].set_title(f'μ={minmax.mean():.3f}, σ={minmax.std():.3f}')
    
    # Z-Score
    zscore_display = np.clip((zscore - zscore.min()) / (zscore.max() - zscore.min()), 0, 1)
    axes[0, 2].imshow(zscore_display)
    axes[0, 2].set_title('Z-Score (rescaled for display)')
    axes[0, 2].axis('off')
    axes[1, 2].hist(zscore.flatten(), bins=50, color='red', alpha=0.7)
    axes[1, 2].set_xlabel('Standardized Value')
    axes[1, 2].set_title(f'μ={zscore.mean():.3f}, σ={zscore.std():.3f}')
    
    plt.tight_layout()
    plt.savefig(PROJECT_ROOT / 'docs' / 'assets' / 'normalization_demo.png', dpi=150)
    plt.show()
    print("✅ Saved to: docs/assets/normalization_demo.png")

if DATASET_DIR and sample_images:
    visualize_normalization(sample_images[0])

In [ ]:
# ============================================================
# COMPLETE PREPROCESSING PIPELINE
# ============================================================

class ImagePreprocessor:
    """
    Complete preprocessing pipeline for YOLO training.
    
    Pipeline: Load → Resize → Normalize → (Optional) Augment
    """
    
    def __init__(self, target_size=(640, 640), normalize_method='minmax'):
        self.target_size = target_size  # (H, W)
        self.normalize_method = normalize_method
    
    def resize(self, image):
        """Resize using OpenCV (faster) or NumPy (educational)."""
        # Using OpenCV for speed in production
        return cv2.resize(image, (self.target_size[1], self.target_size[0]))
    
    def normalize(self, image):
        """Apply normalization."""
        if self.normalize_method == 'minmax':
            return normalize_minmax(image)
        elif self.normalize_method == 'zscore':
            return normalize_zscore(image)
        elif self.normalize_method == 'imagenet':
            return normalize_imagenet(image)
        return image / 255.0
    
    def augment(self, image, augment_prob=0.5):
        """Apply random augmentations."""
        if np.random.random() < augment_prob:
            if np.random.random() < 0.5:
                image = horizontal_flip(image)
            if np.random.random() < 0.3:
                factor = np.random.uniform(0.8, 1.2)
                image = adjust_brightness(image, factor)
            if np.random.random() < 0.3:
                factor = np.random.uniform(0.8, 1.2)
                image = adjust_contrast(image, factor)
        return image
    
    def process(self, image_path, augment=False):
        """Full preprocessing pipeline."""
        # Load
        img = np.array(Image.open(image_path).convert('RGB'))
        # Resize
        img = self.resize(img)
        # Augment (before normalization)
        if augment:
            img = self.augment(img)
        # Normalize
        img = self.normalize(img)
        return img

# Test the pipeline
preprocessor = ImagePreprocessor(target_size=(640, 640))
print("✅ ImagePreprocessor class defined")

In [ ]:
# Test preprocessing pipeline
if DATASET_DIR and sample_images:
    original = np.array(Image.open(sample_images[0]))
    processed = preprocessor.process(sample_images[0], augment=False)
    
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    
    axes[0].imshow(original)
    axes[0].set_title(f'Original: {original.shape}')
    axes[0].axis('off')
    
    axes[1].imshow(processed)
    axes[1].set_title(f'Processed: {processed.shape}, range [{processed.min():.2f}, {processed.max():.2f}]')
    axes[1].axis('off')
    
    plt.tight_layout()
    plt.show()
    print(f"\n✅ Original shape: {original.shape}")
    print(f"✅ Processed shape: {processed.shape}")
    print(f"✅ Value range: [{processed.min():.4f}, {processed.max():.4f}]")

## 📝 Summary

### Implemented (NumPy from scratch):
1. **Bilinear Interpolation** for resizing
2. **Min-Max Normalization** [0-1 scaling]
3. **Z-Score Standardization**
4. **Augmentations**: Flip, Brightness, Contrast, Noise, Rotation

### Key Formulas:
| Transform | Formula |
|-----------|--------|
| Bilinear | f(x,y) = (1-α)(1-β)f₀₀ + α(1-β)f₁₀ + (1-α)βf₀₁ + αβf₁₁ |
| Min-Max | x_norm = (x - min) / (max - min) |
| Z-Score | x_std = (x - μ) / σ |
| H-Flip | I[i,j] → I[i, W-1-j] |

### Next: Task 3 - Convert to YOLO format

---

## 📚 Learning Resources
- [Image Interpolation](https://en.wikipedia.org/wiki/Bilinear_interpolation)
- [Data Augmentation Paper](https://arxiv.org/abs/1712.04621)
- [CS231n Preprocessing](https://cs231n.github.io/neural-networks-2/#datapre)

In [ ]:
print("\n" + "="*60)
print("✅ TASK 2 COMPLETE: Data Preprocessing and Augmentation")
print("="*60)
print("\n📋 Implemented (NumPy from scratch):")
print("   ✓ Bilinear interpolation for resizing")
print("   ✓ Min-Max normalization")
print("   ✓ Z-Score standardization")
print("   ✓ ImageNet normalization")
print("   ✓ Horizontal/Vertical flip")
print("   ✓ Brightness/Contrast adjustment")
print("   ✓ Gaussian noise")
print("   ✓ Rotation, Random crop")
print("   ✓ Complete preprocessing pipeline class")
print("\n➡️ Ready for Task 3: Convert Dataset to YOLO Format")